In [3]:
%matplotlib
import pandas as pd
import quandl
import math, datetime
import numpy as np
from sklearn import preprocessing, svm
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import sklearn
import matplotlib.pyplot as plt
from matplotlib import style
style.use('ggplot')
import pickle

Using matplotlib backend: Qt5Agg


In [22]:
quandl.ApiConfig.api_key = "f7bbgyxXg8VFTxvXx1ii"
data = quandl.get('WIKI/GOOGL')

In [23]:
data = data[['Adj. Open','Adj. High','Adj. Low','Adj. Close', 'Adj. Volume']]

In [24]:
data['HL_PCT'] = (data['Adj. High'] - data['Adj. Close']) / data['Adj. Close'] * 100.0
data['PCT_change'] = (data['Adj. Close'] - data['Adj. Open']) / data['Adj. Open'] * 100.0
data = data[['Adj. Close', 'HL_PCT', 'PCT_change', 'Adj. Volume']]

In [25]:
forecast_col = 'Adj. Close'
data.fillna(-9999, inplace = True)

In [26]:
forecast_out = int(math.ceil(0.1*len(data)))
#math.cel rounds the value to nearset number
#math.cel will return float
#doing 0.1*len(data) we are going to use 10% of the dataframe to predict the value
#print(forecast_out)

In [27]:
data['label'] = data[forecast_col].shift(-forecast_out)
#shift shifts the numbe of columns from the bottom to the top
data.dropna(inplace=True)
#print(len(data))

In [28]:
X = np.array(data.drop(['label'],1))
X = preprocessing.scale(X)
X_lately = X[-forecast_out:]
X = X[:-forecast_out]
data.dropna(inplace=True)
y = np.array(data['label'])
y = y[:-forecast_out]
#print(len(X))
#print(len(y))

In [29]:
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, train_size= 0.2)

In [14]:
clf = LinearRegression(n_jobs=-1)
clf.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=-1, normalize=False)

In [15]:
#with open('../PickleFolder/linearregression.pickle', 'wb') as f:
#    pickle.dump(clf, f)

In [30]:
pickle_in = open('../PickleFolder/linearregression.pickle', 'rb')
clf = pickle.load(pickle_in)

In [31]:
accuracy = clf.score(X_test,y_test)
print (accuracy)

0.7820824382501477


In [32]:
forecast_set = clf.predict(X_lately)

In [33]:
#print(forecast_set, accuracy, forecast_out)

In [34]:
data['Forecast'] = np.nan
last_date = data.iloc[-1].name
#print(last_date)
last_unix = last_date.timestamp()
#print(last_unix)
one_day = 86400
next_unix = last_unix + one_day
#print(next_unix)

In [35]:
for i in forecast_set:
    next_date = datetime.datetime.fromtimestamp(next_unix)
    #print(next_date)
    next_unix += one_day
    #print(next_unix)
    data.loc[next_date] = [np.nan for _ in range(len(data.columns)-1)] + [i]
    #print(data.loc[next_date])
    
#print(data.tail(500))
data['Adj. Close'].plot()
data['Forecast'].plot()
plt.legend(loc=4)
plt.xlabel('Date')
plt.ylabel('Price')
plt.show()